In [3]:
config = """
models:
  - model: flammenai/Mahou-1.3-llama3-8B
    parameters:
      weight: 1.0
  - model: Danielbrdz/Barcenas-Llama3-8b-ORPO
    parameters:
      weight: 1.0
  - model: Weyaxi/Einstein-v6.1-Llama3-8B
    parameters:
      weight: 1.0
merge_method: linear
tokenizer_source: union
dtype: float16"""

In [5]:
with open('config.yaml','w',encoding='utf-8') as f:
    f.write(config)

In [7]:
!git clone https://github.com/arcee-ai/mergekit.git
%cd mergekit

fatal: destination path 'mergekit' already exists and is not an empty directory.
/Users/naveenpoliasetty/Merging /mergekit


In [10]:
OUTPUT_PATH = "models/llama-V1/merged/mini"
LORA_MERGE_CACHE = "/tmp"
CONFIG_YAML = None

In [ ]:
import torch
import yaml

from mergekit.config import MergeConfiguration
from mergekit.merge import MergeOptions, run_merge

with open('config.yaml','r',encoding='utf-8')as f:
    merge_config = MergeConfiguration.model_validate(yaml.safe_load(f))
run_merge (
    merge_config,
    out_path = OUTPUT_PATH,
    options = MergeOptions(
        lora_merge_cache = LORA_MERGE_CACHE,
        copy_tokenizer = True,
        lazy_unpickle = False,
        low_cpu_memory = False),
        )
print("DONE")

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch
from warnings import filterwarnings
filterwarnings("ignore")

QUANT_CONFIG = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

tokenizer = AutoTokenizer.from_pretrained("models/llama-V1/merged/mini")
model = AutoModelForCausalLM.from_pretrained("models/llama-V1/merged/mini", quantization_config=QUANT_CONFIG)

user_message = "Write a recursive function that calculates Fibonacci sequence in Python."
prompt = f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{user_message}

### Response:
"""

inputs = tokenizer(prompt, return_tensors="pt")
outputs = model.generate(**inputs.to("cuda"),
                              max_new_tokens=512,
                              num_beams=10,
                              early_stopping=True,
                              no_repeat_ngram_size=2,
                              )
result = tokenizer.batch_decode(outputs, skip_special_tokens=True)
print(result[0])
     

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
model.push_to_hub("Naveenpoliasetty/llama3-8B-merged-V-small")
tokenizer.push_to_hub("Naveenpoliasetty/llama3-8B-merged-V-small")